## 什么是数据连接？

LLM应用往往需要用户特定的数据，而这些数据并不属于模型的训练集。`LangChain` 的数据连接概念，
通过提供以下组件，实现用户数据的加载、转换、存储和查询：

- 文档加载器：从不同的数据源加载文档
- 文档转换器：拆分文档，将文档转换为问答格式，去除冗余文档，等等
- 文本嵌入模型：将非结构化文本转换为浮点数数组表现形式，也称为向量
- 向量存储：存储和搜索嵌入数据（向量）
- 检索器：提供数据查询的通用接口

我们通过下一段落的实践，来介绍这些组件的使用。

## 数据连接实践

在LLM应用连接用户数据时，通常我们会以如下步骤完成：
1. 加载文档
2. 拆分文档
3. 向量化文档分块
4. 向量数据存储

### 加载文档

`Langchain` 提供了多种文档加载器，用于从不同的数据源加载不同类型的文档。
比如，我们可以从本地文件系统加载文档，也可以通过网络加载远程数据。想了解 `Langchain` 所支持的所有文档加载器，
请参考[Document Loaders](https://python.langchain.com/docs/integrations/document_loaders/)。

在本节课程中，我们将使用最基本的 `TextLoader` 来加载本地文件系统中的文档。

In [3]:
from langchain.document_loaders import TextLoader

loader = TextLoader("./LongText.md", encoding="utf-8")
docs = loader.load()

在上述代码中，我们使用 `TextLoader` 加载了本地文件系统中的 `./README.md` 文件。
`TextLoader` 的 `load` 方法返回一个 `Document` 对象数组（`Document` 是 `Langchain` 提供的文档类，
包含原始内容和元数据）。我们可以通过 `Document` 对象的 `content` 属性来访问文档的原始内容。

In [11]:
print(type(docs))

print(type(docs[0]))
print(dir(docs[0]))

<class 'list'>
<class 'langchain.schema.document.Document'>
['Config', '__abstractmethods__', '__annotations__', '__class__', '__class_vars__', '__config__', '__custom_root_type__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__exclude_fields__', '__fields__', '__fields_set__', '__format__', '__ge__', '__get_validators__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__include_fields__', '__init__', '__init_subclass__', '__iter__', '__json_encoder__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__post_root_validators__', '__pre_root_validators__', '__pretty__', '__private_attributes__', '__reduce__', '__reduce_ex__', '__repr__', '__repr_args__', '__repr_name__', '__repr_str__', '__rich_repr__', '__schema_cache__', '__setattr__', '__setstate__', '__signature__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__try_update_forward_refs__', '__validators__', '_abc_impl', '_calculate_keys', '_copy_and_set_values', '_decompose_class', '_e

In [12]:
docs

[Document(page_content='---\ntitle: 01. Hello Langchain\ntags:\n  - openai\n  - llm\n  - langchain\n---\n\n# WTF Langchain极简入门: 01. Hello Langchain\n\n最近在学习Langchain框架，顺手写一个“WTF Langchain极简入门”，供小白们使用（编程大佬可以另找教程）。本教程默认以下前提：\n- 使用Python版本的[Langchain](https://github.com/hwchase17/langchain)\n- LLM使用OpenAI的模型\n- Langchain目前还处于快速发展阶段，版本迭代频繁，为避免示例代码失效，本教程统一使用版本 **0.0.235**\n\n根据Langchain的[代码约定](https://github.com/hwchase17/langchain/blob/v0.0.235/pyproject.toml#L14C1-L14C24)，Python版本 ">=3.8.1,<4.0"。\n\n推特：[@verysmallwoods](https://twitter.com/verysmallwoods)\n\n所有代码和教程开源在github: [github.com/sugarforever/wtf-langchain](https://github.com/sugarforever/wtf-langchain)\n\n-----\n\n## Langchain 简介\n\n大型语言模型（LLM）正在成为一种具有变革性的技术，使开发人员能够构建以前无法实现的应用程序。然而，仅仅依靠LLM还不足以创建一个真正强大的应用程序。它还需要其他计算资源或知识来源。\n\n`Langchain` 旨在帮助开发这些类型应用程序，比如：\n- 基于文档数据的问答\n- 聊天机器人\n- 代理\n\n## OpenAI 简介\n\n`OpenAI` 是LLM生态的模型层最大的玩家之一。大家目前熟知的 *GPT-3.5*，*GPT-4* 等模型都是OpenAI的产品。它的API允许开发人员通过简单的API调用来访问这些模型。\n\n## Langchain与OpenAI\n\n`Lang

### 拆分文档

拆分文档是文档转换中最常见的操作。拆分文档的目的是将文档拆分为更小的文档块，以便更好地利用模型。
当我们要基于长篇文本构建QA应用时，必须将文本分割成块，这样才能在数据查询中，基于相似性找到与问题最接近的文本块。
这也是常见的AI问答机器人的工作原理。

`Langchain` 提供了多种文档拆分器，用于将文档拆分为更小的文档块。我们逐个看看这些拆分器的使用方法。

#### 按字符拆分

`CharacterTextSplitter` 是最简单的文档拆分器，它将文档拆分为固定长度的文本块。代码如下：

In [15]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(
    separator = "\n\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
)

split_docs = text_splitter.split_documents(docs)
print(len(docs[0].page_content))
print(len(split_docs))

sum = 0
for split_doc in split_docs:
    cal = len(split_doc.page_content)
    print(cal)
    sum += cal
print("sum = ", sum)

3324
5
996
971
872
913
296
sum =  4048


#### 拆分代码

`RecursiveCharacterTextSplitter` 的 `from_language` 函数，可以根据编程语言的特性，
将代码拆分为合适的文本块。代码如下：

In [16]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, Language

PYTHON_CODE = """
def hello_langchain():
    print("Hello, Langchain!")

# Call the function
hello_langchain()
"""
python_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON, chunk_size=50, chunk_overlap=0
)
python_docs = python_splitter.create_documents([PYTHON_CODE])
python_docs

[Document(page_content='def hello_langchain():', metadata={}),
 Document(page_content='print("Hello, Langchain!")', metadata={}),
 Document(page_content='# Call the function\nhello_langchain()', metadata={})]

#### Markdown文档拆分

`MarkdownHeaderTextSplitter` 可以将Markdown文档按照段落结构，基于Markdown语法进行文档分块。代码如下：

In [17]:
from langchain.text_splitter import MarkdownHeaderTextSplitter

markdown_document = "# Chapter 1\n\n    ## Section 1\n\nHi this is the 1st section\n\nWelcome\n\n ### Module 1 \n\n Hi this is the first module \n\n ## Section 2\n\n Hi this is the 2nd section"

headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]

splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
splits = splitter.split_text(markdown_document)

splits

[Document(page_content='Hi this is the 1st section  \nWelcome', metadata={'Header 1': 'Chapter 1', 'Header 2': 'Section 1'}),
 Document(page_content='Hi this is the first module', metadata={'Header 1': 'Chapter 1', 'Header 2': 'Section 1', 'Header 3': 'Module 1'}),
 Document(page_content='Hi this is the 2nd section', metadata={'Header 1': 'Chapter 1', 'Header 2': 'Section 2'})]

#### 按字符递归拆分

这也是对于普通文本的推荐拆分方式。它通过一组字符作为参数，按顺序尝试使用这些字符进行拆分，直到块的大小足够小为止。
默认的字符列表是["\n\n", "\n", " ", ""]。它尽可能地保持段落，句子，单词的完整，因此尽可能地保证语义完整。

In [18]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 100,
    chunk_overlap  = 20,
    length_function = len,
)
texts = text_splitter.split_documents(docs)
print(len(docs[0].page_content))
for split_doc in texts:
  print(len(split_doc.page_content))

3324
73
40
71
81
78
99
30
15
56
17
86
22
91
69
97
95
64
65
77
57
75
98
18
34
99
26
4
56
76
56
96
61
60
81
14
92
57
54
95
87
81
77
76
87
31
78
36
91
21
95
93
13
90


#### 按token拆分

语言模型，例如OpenAI，有token限制。在API调用中，不应超过token限制。
看来，在将文本分块时，根据token数来拆分也是不错的主意。有许多令牌化工具，因此当计算文本的token数时，
应该使用与语言模型相同的令牌化工具。

注，OpenAI所使用的是[tiktoken](https://github.com/openai/tiktoken)。

In [20]:
# !pip install -q tiktoken

^C


In [25]:
import os
os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:7890'
os.environ["HTTP_PROXY"] = 'http://127.0.0.1:7890'

from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=100, chunk_overlap=0
)
split_docs = text_splitter.split_documents(docs)

print(len(split_docs))
split_docs

Created a chunk of size 258, which is longer than the specified 100
Created a chunk of size 188, which is longer than the specified 100
Created a chunk of size 136, which is longer than the specified 100
Created a chunk of size 111, which is longer than the specified 100
Created a chunk of size 231, which is longer than the specified 100
Created a chunk of size 194, which is longer than the specified 100
Created a chunk of size 342, which is longer than the specified 100


26


[Document(page_content='---\ntitle: 01. Hello Langchain\ntags:\n  - openai\n  - llm\n  - langchain\n---\n\n# WTF Langchain极简入门: 01. Hello Langchain', metadata={'source': './LongText.md'}),
 Document(page_content='最近在学习Langchain框架，顺手写一个“WTF Langchain极简入门”，供小白们使用（编程大佬可以另找教程）。本教程默认以下前提：\n- 使用Python版本的[Langchain](https://github.com/hwchase17/langchain)\n- LLM使用OpenAI的模型\n- Langchain目前还处于快速发展阶段，版本迭代频繁，为避免示例代码失效，本教程统一使用版本 **0.0.235**', metadata={'source': './LongText.md'}),
 Document(page_content='根据Langchain的[代码约定](https://github.com/hwchase17/langchain/blob/v0.0.235/pyproject.toml#L14C1-L14C24)，Python版本 ">=3.8.1,<4.0"。', metadata={'source': './LongText.md'}),
 Document(page_content='推特：[@verysmallwoods](https://twitter.com/verysmallwoods)\n\n所有代码和教程开源在github: [github.com/sugarforever/wtf-langchain](https://github.com/sugarforever/wtf-langchain)\n\n-----\n\n## Langchain 简介', metadata={'source': './LongText.md'}),
 Document(page_content='大型语言模型（LLM）正在成为一种具有变革性的技术，使开发人员能够构建以前无法实现的应用程序。然而，仅仅依靠

## 向量化文档分块

`Langchain` 的 `Embeddings` 类实现与文本嵌入模型进行交互的标准接口。
当前市场上有许多嵌入模型提供者（如OpenAI、Cohere、Hugging Face等）。

嵌入模型创建文本片段的向量表示。这意味着我们可以在向量空间中处理文本，并进行语义搜索等操作，
从而查找在向量空间中最相似的文本片段。

注，文本之间的相似度由其向量表示的欧几里得距离来衡量。欧几里得距离是最常见的距离度量方式，也称为L2范数。
对于两个n维向量a和b，欧几里得距离可以通过以下公式计算：

```math
d(a, b) = √((a₁ - b₁)² + (a₂ - b₂)² + ... + (aₙ - bₙ)²)
```

当使用OpenAI的文本嵌入模型时，我们使用如下代码来创建文本片段的向量表示：

In [27]:
import openai
from langchain.embeddings import OpenAIEmbeddings
from dotenv import find_dotenv, load_dotenv
# 如果你设置的是全局的环境变量，这行代码则没有任何作用。
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

embeddings_model = OpenAIEmbeddings()
embeddings = embeddings_model.embed_documents(
    [
        "你好!",
        "Langchain!",
        "你真棒！"
    ]
)
embeddings

[[0.0018379163522136942,
  -0.01672440793886407,
  0.009549586254241358,
  -0.02442976126184427,
  -0.04936479110548523,
  0.011463293056706009,
  -0.008261149880543663,
  -0.010825391099658655,
  -0.01813916145801094,
  -0.013023311479383941,
  0.03410566652417998,
  0.0024679238369164458,
  -0.006492707981610077,
  -0.008930632055275898,
  -0.008002199359843825,
  -0.022509739160900714,
  0.0285729677303965,
  -0.019414965372105648,
  0.011867508480936912,
  -0.020501293568026595,
  -0.004433736275614362,
  0.00697902973664746,
  -0.0001944101853946825,
  0.005248482888216368,
  -0.004819004407424666,
  -0.0009418532285687037,
  0.004389525460971669,
  -0.013263314242001887,
  0.007149558162400619,
  -0.030189829427320115,
  0.03269091300503311,
  0.007616931693694814,
  -0.01811389840145014,
  0.001656335339215186,
  0.01380647833996236,
  -0.01844232186203088,
  -0.007496930312385842,
  -0.0012244881558325996,
  0.0204255043983442,
  0.0054411167212908736,
  0.015195968802548431,
 

In [33]:
print(type(embeddings[0]))
print(type(embeddings[0][0]))
print(len(embeddings))
print(len(embeddings[0]))

<class 'list'>
<class 'float'>
3
1536


### 向量数据存储

向量数据存储，或成为向量数据库，负责存储文本嵌入的向量表现，并提供向量检索的能力。
`Langchain` 提供了多种开源或商业向量数据存储，包括：Chroma, FAISS, Pinecone等。这里以Chroma为例。

`Langchain` 提供了 `Chroma` 包装类，封装了chromadb的操作。

### 存储
在进行以下代码执行前，需要安装 `Chroma` 的包：（安装指定版本，否则会报错）

In [34]:
# !pip install chromadb==0.4.15

In [38]:
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(docs)
print((documents))

[Document(page_content='---\ntitle: 01. Hello Langchain\ntags:\n  - openai\n  - llm\n  - langchain\n---\n\n# WTF Langchain极简入门: 01. Hello Langchain\n\n最近在学习Langchain框架，顺手写一个“WTF Langchain极简入门”，供小白们使用（编程大佬可以另找教程）。本教程默认以下前提：\n- 使用Python版本的[Langchain](https://github.com/hwchase17/langchain)\n- LLM使用OpenAI的模型\n- Langchain目前还处于快速发展阶段，版本迭代频繁，为避免示例代码失效，本教程统一使用版本 **0.0.235**\n\n根据Langchain的[代码约定](https://github.com/hwchase17/langchain/blob/v0.0.235/pyproject.toml#L14C1-L14C24)，Python版本 ">=3.8.1,<4.0"。\n\n推特：[@verysmallwoods](https://twitter.com/verysmallwoods)\n\n所有代码和教程开源在github: [github.com/sugarforever/wtf-langchain](https://github.com/sugarforever/wtf-langchain)\n\n-----\n\n## Langchain 简介\n\n大型语言模型（LLM）正在成为一种具有变革性的技术，使开发人员能够构建以前无法实现的应用程序。然而，仅仅依靠LLM还不足以创建一个真正强大的应用程序。它还需要其他计算资源或知识来源。\n\n`Langchain` 旨在帮助开发这些类型应用程序，比如：\n- 基于文档数据的问答\n- 聊天机器人\n- 代理\n\n## OpenAI 简介\n\n`OpenAI` 是LLM生态的模型层最大的玩家之一。大家目前熟知的 *GPT-3.5*，*GPT-4* 等模型都是OpenAI的产品。它的API允许开发人员通过简单的API调用来访问这些模型。\n\n## Langchain与OpenAI\n\n`Lang

### 检索

In [40]:
db = Chroma.from_documents(documents, OpenAIEmbeddings())

C:\Users\Administrator\.cache\chroma\onnx_models\all-MiniLM-L6-v2\onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:09<00:00, 9.07MiB/s]


TypeError: query() got an unexpected keyword argument 'embedding_function'

In [ ]:
query = "什么是WTF Langchain？"
docs = db.similarity_search(query)
docs

In [ ]:
docs = db.similarity_search_with_score(query)
docs